# Main Error Identification

**Purpose:** Identify errors in final SQL output

---
**Copyright (c) 2025 Michael Powers**


In [2]:
# run each test through gemini
# prompt returns json
# each model gets a dataframe, row for each question, column for each factor


In [3]:
import os
import logging
import json
import pandas as pd
from datetime import datetime
import google.generativeai as genai
import time
import random

In [4]:
gemini_model = 'gemini-2.5-flash-lite-preview-06-17'
api_key="YOUR_API_KEY"

In [5]:
def ask_gemini_json(prompt, use_json=True, model='models/gemini-2.0-flash-lite'):
    import os
    import google.generativeai as genai
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(model)
    if use_json:
        generation_config = genai.GenerationConfig(response_mime_type="application/json")
        response = model.generate_content(prompt, generation_config=generation_config)
    else:
        response = model.generate_content(prompt)
    return response.text

In [6]:
def clean_response(response):
    response = response.replace("<think>", "").replace("</think>", "")
    response = response.strip()
    if response.startswith("```json") and response.endswith("```"):
        response = response[len("```json"): -len("```")].strip()
    elif response.startswith("```") and response.endswith("```"):
        response = response[len("```"): -len("```")].strip()
    if response.lower().startswith('sql'):
        response = response[3:].strip()
    return response

In [7]:
def get_prompt(gold, test):
    example_json = """
    ```json
{
  "syntax_errors": {
    "contains_error": true,
    "details": [
      {
        "description": "...",
      }
    ]
  },
  "linking_errors": {
    "contains_error": false,
    "details": []
  },
  "missing_clauses": {
    "contains_error": true,
    "details": [
      {
        "description": "...",
      }
    ]
  },
  "incorrect_conditions": {
    "contains_error": false,
    "details": []
  },
  "incorrect_aggregations": {
    "contains_error": true,
    "details": [
      {
        "description": "...",
      }
    ]
  },
  "incorrect_joins": {
    "contains_error": true,
    "details": [
      {
        "description": "...",
      }
    ]
  },
  "nested_cte_errors": {
    "contains_error": false,
    "details": []
  },
  "ambiguity_handling_issues": {
    "contains_error": false,
    "details": []
  },
  "dialect_issues": {
    "contains_error": false,
    "details": []
  }
}
```
    """
    
    
    prompt = f"""
You are an expert SQL query analyzer. Your task is to meticulously compare two SQL queries: a "Gold Standard Query" (the correct and desired output) and a "Test Query" (an attempt that may contain errors). The purpose is to identify and explain any discrepancies or errors in the Test Query, categorizing them according to the provided error types.
For each identified error, you must provide a detailed explanation in the description field.

Error Categories to Identify:

**Syntax Errors:**

* Misspellings (e.g., SELCT instead of SELECT, FROMM instead of FROM)

* Incorrect punctuation or special characters (e.g., missing commas, unbalanced parentheses)

* Invalid table or column references (e.g., nonexistent_table.column, table.nonexistent_column)

**Linking Errors:**

* Wrong table used (e.g., querying customers table when users table is required)

* Wrong column used (e.g., selecting product_name when item_description is needed)

* Missing necessary joins (e.g., tables that should be connected for correct data retrieval are not joined)

**Missing Clauses:**

* WHERE clause missing when filtering conditions are required

* GROUP BY clause missing when aggregation needs grouping

* ORDER BY clause missing when specific sorting is required

* HAVING clause missing when filtering on aggregated results is needed

**Incorrect Conditions:**

* Wrong operator used (e.g., = instead of >, <>, LIKE, IN)

* Incorrect value in condition (e.g., status = 'active' instead of status = 'pending')

* Logical errors in AND/OR combinations.

**Incorrect Aggregations:**

* Wrong aggregate function used (e.g., SUM() instead of COUNT(), AVG() instead of MAX())

* Aggregate function applied to the wrong column (e.g., COUNT(order_id) when COUNT(DISTINCT customer_id) is needed)

**Incorrect Joins:**

* Wrong join type (e.g., LEFT JOIN instead of INNER JOIN, FULL OUTER JOIN instead of LEFT JOIN)

* Incorrect join condition (e.g., ON a.id = b.another_id when it should be ON a.id = b.id)

* Missing join (covered in Linking Errors, but explicitly mention if it's the type of join that's the error)

**Nested Queries or Common Table Expression (CTE) Errors:**

* Incorrect structure or logic within subqueries or CTEs

* Missing or misplaced WITH clauses for CTEs

* Incorrect column references within nested structures.

**Ambiguity Handling Issues:**

* The Test Query's interpretation of ambiguous natural language intent differs from the Gold Standard's. Explain why the natural language might be ambiguous and how the Gold Standard resolves it.

**Dialect Issues:**

* The Test Query is syntactically correct in one SQL dialect (e.g., PostgreSQL) but incorrect for the Gold Standard's dialect (e.g., SQL Server, MySQL). Specify the assumed dialect for the Gold Standard and explain the dialect-specific error.


############
**Gold Standard Query:**
{gold}

**Test Query:**
{test}
############
**Output format:**
Your output must be a JSON object. The JSON should contain a key for each of the 9 error categories listed above. Each error category key will map to an object with two keys:

* contains_error: (boolean) true if this error type is present in the Test Query, false otherwise.

* details: (array of objects) If contains_error is true, this array will contain one or more objects, each describing a specific instance of that error type. Each detail object must have the following key:

*  description: (string) Detailed explanation of the error.

Example:
{example_json}
    """
    return prompt

In [8]:
def read_string(filename):
    with open(filename) as f:
        return f.read()

In [9]:
def process_SQL(test_directory="./results/",
                                 gold_directory="./gold/",
                                output_filepath="./results.csv",
                                model='models/gemini-2.0-flash-lite',
                                rpm_limit=15):


    RPM_LIMIT = rpm_limit
    MAX_RETRIES = 5
    BASE_SLEEP_TIME = 8.5
    df_rows = []

    print(f"Starting generation.")
    #Loop through all files in test_directory
    for item in os.listdir(test_directory):
        item_path = os.path.join(test_directory, item)
        if os.path.isfile(item_path):
            test_filename = item_path
            gold_filename = os.path.join(gold_directory, item)
            test_contents = read_string(test_filename)
            gold_contents = read_string(gold_filename)
            prompt = get_prompt(gold_contents, test_contents)
            
            # Counter for API calls made within the current minute
            requests_in_minute = 0
            start_time_minute = time.time()
            retries = 0
        
            while retries < MAX_RETRIES:
                # Check RPM limit
                current_time = time.time()
                if current_time - start_time_minute >= 60:
                    requests_in_minute = 0
                    start_time_minute = current_time

                if requests_in_minute >= RPM_LIMIT:
                    wait_time = 60 - (current_time - start_time_minute)
                    print(f"Rate limit hit. Waiting for {wait_time:.2f} seconds...")
                    time.sleep(wait_time + 1)
                    requests_in_minute = 0
                    start_time_minute = time.time()

                try: # LLM CALL
                    print(f'Making LLM Call')
                    response = ask_gemini_json(prompt, use_json=True, model=model)
                    requests_in_minute += 1
                
                    # Clean the response string
                    response = clean_response(response)

                    # CONVERT OR SAVE OR WHATEVER WE ARE DOING GOES HERE
                    #print('!!!!!!!!')
                    try:
                        data = json.loads(response)
                        for error_type, error_info in data.items():
                            contains_error = error_info['contains_error']
                            details_descriptions = [detail['description'] for detail in error_info['details']]
                            details_str = "; ".join(details_descriptions) if details_descriptions else ""
                            df_rows.append({
                                "ID" : item,
                                "Error Type": error_type,
                                "Contains Error": contains_error,
                                "Details": details_str
                            })
                    except Exception as e:
                        print(f'Error decoding json response: {e}')
                        print("-------DATA NO GOOD")
                        print("!!!CANCEL - Try Again!!!")
                        return
                        
                    break # Success, break out of retry loop
                except Exception as e:
                    retries += 1
                    sleep_duration = BASE_SLEEP_TIME * (2 ** (retries - 1)) + random.uniform(0, 1)
                    print(f"API Error : {e}. Retrying in {sleep_duration:.2f}s... (Attempt {retries}/{MAX_RETRIES})")
                    time.sleep(sleep_duration)

                if retries == MAX_RETRIES:
                    print(f"Failed to process review from input line {i} after {MAX_RETRIES} retries. Skipping.")
                    print("-------DATA NO GOOD")
                    print("!!!CANCEL - Try Again!!!")
                    return
    df = pd.DataFrame(df_rows)
    print(df)
    df.to_csv(output_filepath, index=False)
    print("------DONE-----")
    return df
    

In [14]:
test_directory = "../../spider/Spider2-main/spider2-lite/evaluation_suite/gemini_flash_v4c_submission/"
gold_directory = "../../spider/Spider2-main/spider2-lite/evaluation_suite/gold/sql/"
output_filename = "./gemini-flash-v4f-score.csv"
current_df = process_SQL(test_directory, gold_directory, output_filename, model=gemini_model)

Starting generation.
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
API Error : 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 14
}
]. Retrying in 8.65s... (Attempt 1/5)
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
Making LLM Call
               ID                 Error Type  Contains Error  \
0    local301.sql              syntax_errors           False   
1    local301.sql             linking_errors         

In [15]:
df_filename = "./gemini-flash-v4f-score.csv"
df = pd.read_csv(df_filename)
df['Contains Error'] = df['Contains Error'].astype(bool)
error_count = df['Contains Error'].sum()
error_type_counts = df.groupby('Error Type')['Contains Error'].sum()
print(f'Total Errors: {error_count}')
print(error_type_counts)

Total Errors: 96
Error Type
ambiguity_handling_issues     2
dialect_issues                4
incorrect_aggregations       22
incorrect_conditions          8
incorrect_joins              20
linking_errors               19
missing_clauses              13
nested_cte_errors             7
syntax_errors                 1
Name: Contains Error, dtype: int64


In [ ]:
#seperately evaluate the RAG
